# 🍺 기네스의 비밀: Student's t-분포의 탄생

## 📖 1908년 더블린의 이야기

1908년, 아일랜드 더블린의 **세인트 제임스 게이트 양조장**에서 일하던 젊은 화학자가 있었습니다. 그의 이름은 **William Sealy Gosset**. 🧑‍🔬

매일 아침, Gosset은 고민에 빠져있었습니다:

> "맥주의 품질을 확인하려면 많은 샘플이 필요한데... 😰  
> 하루에 겨우 3-4병만 검사할 수 있어.  
> Z-검정은 σ를 알아야 하는데, 우리는 표본표준편차 s만 있고...  
> 도대체 어떻게 해야 할까?"

이 고민이 바로 **통계학 역사상 가장 중요한 발견** 중 하나로 이어지게 됩니다! 🌟

---

## 🎯 학습 목표

이 노트북에서 우리는:
1. **작은 표본 문제**의 본질을 이해합니다
2. **t-분포의 탄생 배경**을 스토리로 따라갑니다
3. **몬테카를로 시뮬레이션**으로 t-분포를 직접 유도합니다
4. **자유도의 의미**를 시각적으로 탐구합니다
5. **Z-검정 vs t-검정**의 차이를 체감합니다

In [1]:
# 필요한 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, IntSlider
from IPython.display import display, clear_output
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")

print("🍺 기네스 양조장으로의 시간여행 준비 완료!")
print("📅 1908년 더블린으로 떠납니다...")

🍺 기네스 양조장으로의 시간여행 준비 완료!
📅 1908년 더블린으로 떠납니다...


## 🧪 1. Gosset의 문제: 작은 표본의 딜레마

### 🏭 기네스 양조장의 현실

Gosset이 직면한 현실적 제약들:

1. **비용 문제** 💰
   - 맥주 한 병을 검사하면 판매할 수 없음
   - 많은 샘플 = 큰 손실

2. **시간 제약** ⏰
   - 화학 분석에는 시간이 오래 걸림
   - 하루에 3-4개 샘플이 한계

3. **알려지지 않은 σ** ❓
   - 새로운 배치마다 품질이 다름
   - 과거 데이터로부터 σ를 정확히 알 수 없음

### 🤔 핵심 질문

> **σ 대신 s(표본표준편차)를 사용하면 어떻게 될까?**

$$Z = \frac{\bar{X} - \mu}{\sigma/\sqrt{n}} \quad \text{vs} \quad T = \frac{\bar{X} - \mu}{s/\sqrt{n}}$$

좌측은 **알려진 σ** (완벽한 세상), 우측은 **추정된 s** (현실 세상)

### 📊 t-통계량이란?

**t-통계량**은 단순히 **계산 방법**입니다:
- Z-통계량에서 **σ 대신 s(표본표준편차)**를 사용한 것
- 공식: $t = \frac{\bar{X} - \mu_0}{s/\sqrt{n}}$

**⚠️ 중요한 구분:**
- **t-통계량**: 계산 방식 (σ → s로 바꾼 것)
- **t-분포**: 그 t-통계량들이 실제로 따르는 수학적 분포

즉, t-통계량을 계산하는 것과 t-분포는 **별개의 개념**입니다!
- 계산은 간단하지만, 그 결과가 어떤 분포를 따르는지는 별도의 수학적 발견이었습니다.

In [2]:
# 🍺 기네스 맥주 품질 검사 시뮬레이션
def guinness_quality_simulation(n_samples=4, true_alcohol=5.0, true_std=0.3, n_days=1000):
    """기네스 양조장의 일일 품질 검사 시뮬레이션"""
    
    np.random.seed(42)
    
    daily_means = []
    daily_stds = []
    z_statistics = []  # σ를 알 때
    t_statistics = []  # σ를 모를 때 (s 사용)
    
    print(f"🍺 기네스 양조장 품질 검사 시뮬레이션")
    print(f"📊 실제 알코올 도수: {true_alcohol}% (표준편차: {true_std}%)")
    print(f"🔬 일일 검사 샘플 수: {n_samples}개")
    print(f"📅 시뮬레이션 기간: {n_days}일")
    print("")
    
    for day in tqdm(range(n_days), desc="품질 검사 진행 중"):
        # 하루치 맥주 샘플 검사
        daily_samples = np.random.normal(true_alcohol, true_std, n_samples)
        
        daily_mean = np.mean(daily_samples)
        daily_std = np.std(daily_samples, ddof=1)  # 표본표준편차
        
        daily_means.append(daily_mean)
        daily_stds.append(daily_std)
        
        # Z-통계량 (σ=0.3 알려진 경우)
        z_stat = (daily_mean - true_alcohol) / (true_std / np.sqrt(n_samples))
        z_statistics.append(z_stat)
        
        # t-통계량 (s 사용하는 경우)
        if daily_std > 0:  # 0으로 나누기 방지
            t_stat = (daily_mean - true_alcohol) / (daily_std / np.sqrt(n_samples))
            t_statistics.append(t_stat)
    
    return {
        'daily_means': daily_means,
        'daily_stds': daily_stds,
        'z_statistics': z_statistics,
        't_statistics': t_statistics,
        'n_samples': n_samples,
        'true_alcohol': true_alcohol,
        'true_std': true_std
    }

def visualize_gosset_discovery(simulation_data):
    """Gosset의 발견을 시각화"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            '1. 일일 알코올 도수 측정값',
            '2. 일일 표본표준편차 변동',
            '3. Z vs t 통계량 분포 비교',
            '4. 이론 vs 실제 분포'
        ],
        specs=[[
            {'type': 'scatter'}, {'type': 'histogram'}
        ], [
            {'type': 'histogram'}, {'type': 'scatter'}
        ]]
    )
    
    # 1. 일일 측정값 시계열
    days = list(range(1, len(simulation_data['daily_means']) + 1))
    fig.add_trace(
        go.Scatter(
            x=days[:100],  # 처음 100일만 표시
            y=simulation_data['daily_means'][:100],
            mode='lines+markers',
            name='일일 평균 도수',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    
    # 목표 알코올 도수 선
    fig.add_hline(
        y=simulation_data['true_alcohol'],
        line_dash="dash",
        line_color="red",
        annotation_text=f"목표: {simulation_data['true_alcohol']}%",
        row=1, col=1
    )
    
    # 2. 표본표준편차 히스토그램
    fig.add_trace(
        go.Histogram(
            x=simulation_data['daily_stds'],
            nbinsx=50,
            name='표본표준편차 s',
            marker_color='orange',
            opacity=0.7
        ),
        row=1, col=2
    )
    
    # 실제 σ 선
    fig.add_vline(
        x=simulation_data['true_std'],
        line_dash="dash",
        line_color="red",
        annotation_text=f"실제 σ = {simulation_data['true_std']}",
        row=1, col=2
    )
    
    # 3. Z vs t 통계량 비교
    fig.add_trace(
        go.Histogram(
            x=simulation_data['z_statistics'],
            nbinsx=50,
            name='Z-통계량 (σ 알려짐)',
            marker_color='blue',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Histogram(
            x=simulation_data['t_statistics'],
            nbinsx=50,
            name='t-통계량 (σ 모름)',
            marker_color='red',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    # 4. 이론적 분포 곡선
    x_range = np.linspace(-4, 4, 200)
    
    # 표준정규분포 (Z)
    y_normal = stats.norm.pdf(x_range, 0, 1)
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_normal,
            mode='lines',
            name='표준정규분포 N(0,1)',
            line=dict(color='blue', width=3)
        ),
        row=2, col=2
    )
    
    # t-분포 (자유도 = n-1)
    df = simulation_data['n_samples'] - 1
    y_t = stats.t.pdf(x_range, df)
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_t,
            mode='lines',
            name=f't-분포 (df={df})',
            line=dict(color='red', width=3, dash='dash')
        ),
        row=2, col=2
    )
    
    # 레이아웃 업데이트
    fig.update_layout(
        title=f'🍺 Gosset의 발견: 작은 표본 (n={simulation_data["n_samples"]})에서의 문제',
        height=800,
        showlegend=True
    )
    
    # 축 레이블
    fig.update_xaxes(title_text="일수", row=1, col=1)
    fig.update_yaxes(title_text="알코올 도수 (%)", row=1, col=1)
    fig.update_xaxes(title_text="표본표준편차", row=1, col=2)
    fig.update_xaxes(title_text="통계량 값", row=2, col=1)
    fig.update_xaxes(title_text="값", row=2, col=2)
    fig.update_yaxes(title_text="확률밀도", row=2, col=2)
    
    return fig

# 기네스 품질 검사 시뮬레이션 실행
guinness_data = guinness_quality_simulation(n_samples=4, n_days=1000)
fig = visualize_gosset_discovery(guinness_data)
fig.show()

# 통계 요약
z_stats = guinness_data['z_statistics']
t_stats = guinness_data['t_statistics']

print(f"""
🔍 Gosset의 관찰 결과:

📊 Z-통계량 (σ 알려진 경우):
   평균: {np.mean(z_stats):.3f} (이론값: 0.000)
   표준편차: {np.std(z_stats):.3f} (이론값: 1.000)

📊 t-통계량 (σ 모르는 경우):
   평균: {np.mean(t_stats):.3f} (이론값: 0.000)
   표준편차: {np.std(t_stats):.3f} (이론값: {np.sqrt(3/(4-2)):.3f} for df=3)

🎯 핵심 발견:
   💡 t-통계량이 Z-통계량보다 더 넓게 퍼져있다!
   💡 이는 s를 사용함으로써 생기는 추가적인 불확실성 때문이다
   💡 표본 크기가 작을수록 이 차이가 더 크다
   
📈 표본표준편차 변동성:
   💡 실제 σ = {guinness_data['true_std']}, 평균 s = {np.mean(guinness_data['daily_stds']):.3f}
   💡 s의 표준편차 = {np.std(guinness_data['daily_stds']):.3f} (날마다 이만큼 변동!)

📊 4번 그래프 해석:
   💡 파란 실선: Z-통계량이 따라야 하는 이론적 표준정규분포 N(0,1)
   💡 빨간 점선: t-통계량이 따라야 하는 이론적 t-분포 (df=3)
   💡 t-분포가 표준정규분포보다 더 넓고 꼬리가 두꺼움을 확인!
   💡 3번의 실제 시뮬레이션 결과와 4번의 이론적 분포가 일치함을 보여줌
""")

🍺 기네스 양조장 품질 검사 시뮬레이션
📊 실제 알코올 도수: 5.0% (표준편차: 0.3%)
🔬 일일 검사 샘플 수: 4개
📅 시뮬레이션 기간: 1000일



품질 검사 진행 중: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 107944.82it/s]


🔍 Gosset의 관찰 결과:

📊 Z-통계량 (σ 알려진 경우):
   평균: 0.039 (이론값: 0.000)
   표준편차: 0.985 (이론값: 1.000)

📊 t-통계량 (σ 모르는 경우):
   평균: 0.022 (이론값: 0.000)
   표준편차: 1.845 (이론값: 1.225 for df=3)

🎯 핵심 발견:
   💡 t-통계량이 Z-통계량보다 더 넓게 퍼져있다!
   💡 이는 s를 사용함으로써 생기는 추가적인 불확실성 때문이다
   💡 표본 크기가 작을수록 이 차이가 더 크다

📈 표본표준편차 변동성:
   💡 실제 σ = 0.3, 평균 s = 0.277
   💡 s의 표준편차 = 0.117 (날마다 이만큼 변동!)

📊 4번 그래프 해석:
   💡 파란 실선: Z-통계량이 따라야 하는 이론적 표준정규분포 N(0,1)
   💡 빨간 점선: t-통계량이 따라야 하는 이론적 t-분포 (df=3)
   💡 t-분포가 표준정규분포보다 더 넓고 꼬리가 두꺼움을 확인!
   💡 3번의 실제 시뮬레이션 결과와 4번의 이론적 분포가 일치함을 보여줌



## 🎲 2. 몬테카를로 시뮬레이션: t-분포 탄생의 비밀

### 🔬 몬테카를로 시뮬레이션이란?

Gosset의 발견을 이해하기 위해 **몬테카를로 시뮬레이션**이라는 강력한 도구를 사용해보겠습니다.

#### 🎯 몬테카를로 시뮬레이션의 핵심 아이디어

> **"수학적으로 복잡한 문제를 컴퓨터로 반복 실험하여 해결하자!"**

**1. 반복 실험** 🔄
- 동일한 조건에서 수천 번 실험
- 각 실험은 독립적으로 수행
- 많은 횟수 → 더 정확한 결과

**2. 확률적 샘플링** 🎲
- 랜덤하게 표본을 생성
- 실제 데이터 수집과 동일한 과정
- 무작위성이 핵심!

**3. 통계적 추론** 📊
- 반복된 결과들의 패턴 분석
- 이론적 분포를 경험적으로 발견
- "큰 수의 법칙"을 활용

### 🍺 기네스 양조장에서의 적용

우리가 곧 실행할 시뮬레이션:

**🎯 목표**: 새로운 분포를 몬테카를로로 직접 발견하기

**📋 과정**:
1. **정규분포 N(0,1)에서 표본 생성**
   - 표본 크기: n (예: 5개)
   - 수천 번 반복

2. **각 표본마다 표준화 값 계산**
   ```
   계산값 = (표본평균 - 0) / (표본표준편차 / √n)
   ```
   ⚠️ **이것은 단순한 표준화 계산입니다!**  
   (σ 대신 s를 사용한 표준화)

3. **수천 개의 계산값들 수집**
   - 이들의 히스토그램이 어떤 패턴을 보일까?

4. **놀라운 발견: 새로운 분포!**
   - 정규분포가 아닌 **새로운 모양**의 분포 등장!
   - Gosset이 이 분포를 **"t-분포"**라고 명명했습니다! 🎉

### 🤔 왜 이런 이름이 붙었나?

**역사적 순서:**

1. **1908년 Gosset의 문제**:
   - σ 대신 s를 쓰면 어떻게 될까?
   - 계산: `(x̄ - μ₀)/(s/√n)`

2. **실험과 발견**:
   - 이 계산값들이 정규분포를 따르지 않음!
   - 새로운 분포 패턴 발견

3. **이름 짓기**:
   - 계산 공식: "t-통계량" (σ 대신 s 사용한 표준화)
   - 발견된 분포: "t-분포" (그 계산값들의 분포)

**🔍 혼동의 원인:**
- 이름이 우연히 같아서 헷갈림! 
- 실제로는 **완전히 다른 개념**

### 📐 t-분포의 수학적 정의

**t-분포 (Student's t-distribution)**는 다음과 같이 정의됩니다:

$$t \sim t(\text{df}) \quad \text{where df = n-1}$$

**확률밀도함수:**
$$f(t) = \frac{\Gamma\left(\frac{\text{df}+1}{2}\right)}{\sqrt{\text{df}\pi}\,\Gamma\left(\frac{\text{df}}{2}\right)} \left(1 + \frac{t^2}{\text{df}}\right)^{-\frac{\text{df}+1}{2}}$$

**🔢 감마함수 Γ(x)란?**
- **정의**: $\Gamma(x) = \int_0^{\infty} t^{x-1} e^{-t} dt$ (x > 0)
- **팩토리얼의 일반화**: $\Gamma(n) = (n-1)!$ (n이 양의 정수일 때)
- **특별한 값들**:
  - $\Gamma(1) = 0! = 1$
  - $\Gamma(2) = 1! = 1$
  - $\Gamma(1/2) = \sqrt{\pi}$
- **역할**: t-분포의 정규화 상수 (전체 면적을 1로 만듦)

**주요 특성:**
- **평균**: 0 (df > 1일 때)
- **분산**: $\frac{\text{df}}{\text{df}-2}$ (df > 2일 때)
- **자유도**: df = n-1 (표본크기에서 1을 뺀 값)
- **대칭**: 0을 중심으로 좌우 대칭
- **두꺼운 꼬리**: 표준정규분포보다 꼬리가 두꺼움

### 🎯 시뮬레이션의 진짜 목적

**우리가 실제로 하는 것:**

1. **표준화 계산 반복** 📊
   ```python
   calculated_values = []
   for i in range(10000):
       sample = np.random.normal(0, 1, n)  # 정규분포에서 표본
       # 단순한 표준화 (σ 대신 s 사용)
       value = (np.mean(sample) - 0) / (np.std(sample, ddof=1) / np.sqrt(n))
       calculated_values.append(value)
   
   # 이 값들이 어떤 분포를 가질까?
   히스토그램(calculated_values) ← 새로운 분포 발견!
   ```

2. **분포 패턴 확인** ✅
   ```python
   # 수학자들이 유도한 이론적 t-분포
   theoretical_t_dist = stats.t.pdf(x, df=n-1)
   
   # 비교: 우리가 발견한 패턴 vs 이론
   "와! 똑같은 모양이다!" 🎯
   ```

### 💡 왜 몬테카를로를 사용하는가?

**1. 직관적 이해** 🧠
- "σ 대신 s를 쓰면 어떻게 될까?"의 답을 직접 확인
- 복잡한 수학 없이 실험으로 체험

**2. 역사적 재현** 🕰️
- 1908년 Gosset의 발견 과정을 현대적으로 재현
- "이런 식으로 새로운 분포가 발견되었구나!"

**3. 이론 검증** ✅
- 수학적 공식이 현실과 일치하는지 확인

### 🎯 시뮬레이션에서 발견할 것들

- **표본평균들의 분포**: 정규분포 형태 (중심극한정리)
- **표본표준편차들의 변동**: σ=1 주변에서 흩어짐
- **표준화 값들의 분포**: 바로 이것이 t-분포! **✨**
- **이론과 실제의 일치**: 수학 공식이 현실을 정확히 설명

### 🚀 역사적 의미

> 1908년 Gosset은 **수학적으로** 이 새로운 분포를 발견했습니다.  
> 오늘 우리는 **컴퓨터로** 같은 발견을 재현합니다!

자, 이제 몬테카를로 마법을 시작해보겠습니다! ✨

In [3]:
# 📜 Student의 원리 재현: 몬테카를로로 t-분포 유도하기
def derive_t_distribution_monte_carlo(sample_size=5, n_simulations=10000):
    "몬테카를로 시뮬레이션으로 t-분포 유도"
    
    print(f"🎲 몬테카를로 시뮬레이션으로 t-분포 유도하기")
    print(f"📊 표본 크기: {sample_size}")
    print(f"🔄 시뮬레이션 횟수: {n_simulations:,}회")
    print("")
    
    np.random.seed(42)
    
    # 표준정규분포 N(0,1)에서 표본들을 뽑는다고 가정
    # (일반성을 잃지 않음 - 모든 정규분포는 표준화 가능)
    true_mean = 0
    true_std = 1
    
    t_statistics = []
    sample_means = []
    sample_stds = []
    
    for i in tqdm(range(n_simulations), desc="t-분포 유도 중"):
        # 표본 추출
        sample = np.random.normal(true_mean, true_std, sample_size)
        
        # 표본통계량 계산
        sample_mean = np.mean(sample)
        sample_std = np.std(sample, ddof=1)  # 표본표준편차
        
        # t-통계량 계산
        if sample_std > 0:  # 0으로 나누기 방지
            t_stat = (sample_mean - true_mean) / (sample_std / np.sqrt(sample_size))
            t_statistics.append(t_stat)
        
        sample_means.append(sample_mean)
        sample_stds.append(sample_std)
    
    return {
        't_statistics': t_statistics,
        'sample_means': sample_means,
        'sample_stds': sample_stds,
        'sample_size': sample_size,
        'df': sample_size - 1
    }

def visualize_t_distribution_derivation(monte_carlo_data):
    "몬테카를로로 유도한 t-분포 시각화"
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            '1. 표본평균들의 분포',
            '2. 표본표준편차들의 분포',
            '3. 유도된 t-분포 vs 이론적 t-분포',
            '4. t-분포 vs 표준정규분포'
        ]
    )
    
    # 1. 표본평균들의 분포
    fig.add_trace(
        go.Histogram(
            x=monte_carlo_data['sample_means'],
            nbinsx=50,
            name='표본평균들',
            marker_color='lightblue',
            opacity=0.7,
            histnorm='probability density'
        ),
        row=1, col=1
    )
    
    # 이론적 분포 (표본평균)
    x_mean = np.linspace(-2, 2, 100)
    y_mean_theory = stats.norm.pdf(x_mean, 0, 1/np.sqrt(monte_carlo_data['sample_size']))
    fig.add_trace(
        go.Scatter(
            x=x_mean,
            y=y_mean_theory,
            mode='lines',
            name='이론적 분포',
            line=dict(color='red', width=3)
        ),
        row=1, col=1
    )
    
    # 2. 표본표준편차들의 분포
    fig.add_trace(
        go.Histogram(
            x=monte_carlo_data['sample_stds'],
            nbinsx=50,
            name='표본표준편차들',
            marker_color='lightgreen',
            opacity=0.7
        ),
        row=1, col=2
    )
    
    # 실제 표준편차 선
    fig.add_vline(
        x=1.0,
        line_dash="dash",
        line_color="red",
        annotation_text="σ = 1",
        row=1, col=2
    )
    
    # 3. 유도된 t-분포 vs 이론적 t-분포
    fig.add_trace(
        go.Histogram(
            x=monte_carlo_data['t_statistics'],
            nbinsx=50,
            name='몬테카를로 t-통계량',
            marker_color='orange',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    # 이론적 t-분포
    x_t = np.linspace(-5, 5, 200)
    df = monte_carlo_data['df']
    y_t_theory = stats.t.pdf(x_t, df)
    fig.add_trace(
        go.Scatter(
            x=x_t,
            y=y_t_theory,
            mode='lines',
            name=f'이론적 t-분포 (df={df})',
            line=dict(color='red', width=3)
        ),
        row=2, col=1
    )
    
    # 4. t-분포 vs 표준정규분포 비교
    y_normal = stats.norm.pdf(x_t, 0, 1)
    
    fig.add_trace(
        go.Scatter(
            x=x_t,
            y=y_t_theory,
            mode='lines',
            name=f't-분포 (df={df})',
            line=dict(color='red', width=3)
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=x_t,
            y=y_normal,
            mode='lines',
            name='표준정규분포',
            line=dict(color='blue', width=3, dash='dash')
        ),
        row=2, col=2
    )
    
    # 레이아웃
    fig.update_layout(
        title=f'📜 Student의 원리: 몬테카를로로 t-분포 유도 (n={monte_carlo_data["sample_size"]}, df={df})',
        height=800,
        showlegend=True
    )
    
    return fig

# 인터랙티브 t-분포 유도
@interact(
    sample_size=IntSlider(min=3, max=20, step=1, value=5, description='표본크기 (n)'),
    n_simulations=widgets.Dropdown(
        options=[('빠름 (1,000회)', 1000), ('보통 (5,000회)', 5000), ('정확 (10,000회)', 10000)],
        value=5000,
        description='시뮬레이션:'
    )
)
def interactive_t_derivation(sample_size, n_simulations):
    # 몬테카를로 시뮬레이션
    mc_data = derive_t_distribution_monte_carlo(sample_size, n_simulations)
    
    # 시각화
    fig = visualize_t_distribution_derivation(mc_data)
    fig.show()
    
    # 통계적 검증
    t_stats = mc_data['t_statistics']
    df = mc_data['df']
    
    # 이론값과 비교
    theoretical_mean = 0
    theoretical_var = df / (df - 2) if df > 2 else np.inf
    theoretical_std = np.sqrt(theoretical_var) if df > 2 else np.inf
    
    empirical_mean = np.mean(t_stats)
    empirical_std = np.std(t_stats)
    
    print(f"""
🎯 Student의 발견 검증:

📊 자유도 (df): {df}

📊 이론적 평균: {theoretical_mean:.3f}  |  실제 평균: {empirical_mean:.3f}

📊 이론적 표준편차: {theoretical_std:.3f}  |  실제 표준편차: {empirical_std:.3f}

📐 평균 오차: {abs(theoretical_mean - empirical_mean):.4f}

📐 표준편차 오차: {abs(theoretical_std - empirical_std):.4f}


💡 핵심 통찰:

{'✅ 우리가 몬테카를로로 유도한 분포가 이론적 t-분포와 일치합니다!' if abs(theoretical_std - empirical_std) < 0.1 else '⚠️ 시뮬레이션 횟수를 늘려보세요!'}

🎉 이것이 바로 1908년 Gosset이 발견한 것입니다!

""")

interactive(children=(IntSlider(value=5, description='표본크기 (n)', max=20, min=3), Dropdown(description='시뮬레이션:'…

## 📊 3. 자유도의 비밀: df = n - 1

### 🤔 진짜 문제: 작은 표본의 불확실성

t-분포의 가장 중요한 특징은 **자유도(degrees of freedom, df)**입니다. 하지만 수학적 정의보다 **실제 의미**가 더 중요합니다!

#### 🎯 핵심 아이디어: 표본표준편차의 신뢰성

**상황 비교:**
- **큰 표본 (n=100)**: 표본표준편차 s가 σ에 매우 가까움 ✅
- **작은 표본 (n=3)**: 표본표준편차 s가 σ와 많이 다를 수 있음 ⚠️

### 🧪 구체적 예시로 이해하기

**시나리오**: 진짜 σ = 10인 모집단에서 표본을 뽑는다고 가정

#### 📊 큰 표본 (n=100)의 경우:
```
표본 1: s = 9.8
표본 2: s = 10.1  
표본 3: s = 9.9
표본 4: s = 10.2
→ s가 σ=10 주변에서 안정적!
```

#### 📊 작은 표본 (n=3)의 경우:
```
표본 1: s = 7.2   ← 많이 작음!
표본 2: s = 13.8  ← 많이 큼!
표본 3: s = 9.1
표본 4: s = 11.5
→ s가 σ=10과 많이 다름!
```

### 💡 자유도의 실제 의미

**자유도 = 표본표준편차의 신뢰성을 나타내는 지표**

- **df = n-1이 작을수록**: s가 σ와 많이 다를 가능성 ↑
- **df = n-1이 클수록**: s가 σ에 가까울 가능성 ↑

### 🔗 왜 n-1인가? (수학적 이유)

표본 크기가 n=3인 경우:
- 데이터: [x₁, x₂, x₃]
- 표본평균: $\bar{x} = \frac{x_1 + x_2 + x_3}{3}$

**제약 조건**: 편차들의 합은 항상 0
$$(x_1 - \bar{x}) + (x_2 - \bar{x}) + (x_3 - \bar{x}) = 0$$

**실제 의미:**
- 첫 번째와 두 번째 편차를 알면, 세 번째는 **자동으로 결정**
- **독립적인 정보**는 2개뿐 (= n-1)
- 따라서 **자유도 = 2**

### 🎯 자유도가 t-분포에 미치는 영향

#### 1. **작은 자유도 (df=2, 표본크기=3)**:
- s가 σ와 많이 다를 수 있음
- t-통계량이 **매우 불안정**
- 분포가 **넓고 꼬리가 두꺼움** (불확실성 반영)

#### 2. **큰 자유도 (df=99, 표본크기=100)**:
- s가 σ에 거의 근사
- t-통계량이 **안정적**
- 분포가 **정규분포에 가까워짐**

### 🔥 왜 이게 중요한가?

**실용적 의미:**
- **작은 표본**: 더 보수적으로 판단해야 함 (넓은 신뢰구간)
- **큰 표본**: 더 정확한 판단 가능 (좁은 신뢰구간)

**예시:**
- df=2일 때 95% 임계값: ±4.303 (매우 보수적!)
- df=∞일 때 95% 임계값: ±1.96 (정규분포)

### 📈 자유도의 실제 효과

- **df ↓ (작은 표본)**: 
  - 불확실성 ↑
  - 분포 넓어짐 ↑ 
  - 임계값 커짐 ↑
  - 보수적 판단 ↑

- **df ↑ (큰 표본)**:
  - 불확실성 ↓
  - 분포 좁아짐 ↓
  - 임계값 작아짐 ↓  
  - 정확한 판단 ↑

### 🎉 핵심 통찰

> **자유도 = 우리가 얼마나 확신할 수 있는가의 지표**

작은 표본에서는 겸손해야 하고, 큰 표본에서는 더 자신감을 가질 수 있다는 **통계학의 지혜**입니다!

In [4]:
# 📊 자유도의 영향 시각화
def visualize_degrees_of_freedom(df_list=[1, 2, 3, 5, 10, 30]):
    "자유도가 t-분포에 미치는 영향 시각화"
    
    x = np.linspace(-4, 4, 1000)
    
    fig = go.Figure()
    
    # 각 자유도별 t-분포
    colors = px.colors.qualitative.Set1
    
    for i, df in enumerate(df_list):
        y = stats.t.pdf(x, df)
        fig.add_trace(
            go.Scatter(
                x=x, y=y,
                mode='lines',
                name=f't-분포 (df={df})',
                line=dict(width=2.5, color=colors[i % len(colors)])
            )
        )
    
    # 표준정규분포 추가
    y_normal = stats.norm.pdf(x, 0, 1)
    fig.add_trace(
        go.Scatter(
            x=x, y=y_normal,
            mode='lines',
            name='표준정규분포 N(0,1)',
            line=dict(width=3, color='black', dash='dash')
        )
    )
    
    # 레이아웃
    fig.update_layout(
        title='🔢 자유도(df)가 t-분포에 미치는 영향',
        xaxis_title='값',
        yaxis_title='확률밀도',
        template='plotly_white',
        height=600,
        legend=dict(x=0.7, y=0.95)
    )
    
    return fig

def degrees_of_freedom_comparison(df_values=[1, 3, 10, 30]):
    "자유도별 주요 통계량 비교"
    
    comparison_data = []
    
    for df in df_values:
        # 주요 통계량 계산
        mean = 0  # 항상 0
        
        if df > 2:
            variance = df / (df - 2)
            std = np.sqrt(variance)
        else:
            variance = np.inf
            std = np.inf
        
        # 95% 임계값
        critical_95 = stats.t.ppf(0.975, df)
        
        # 99% 임계값
        critical_99 = stats.t.ppf(0.995, df)
        
        comparison_data.append({
            '자유도 (df)': df,
            '평균': mean,
            '분산': variance,
            '표준편차': std,
            '95% 임계값': critical_95,
            '99% 임계값': critical_99
        })
    
    # 표준정규분포 추가
    comparison_data.append({
        '자유도 (df)': '∞ (정규분포)',
        '평균': 0,
        '분산': 1,
        '표준편차': 1,
        '95% 임계값': 1.96,
        '99% 임계값': 2.576
    })
    
    df_comparison = pd.DataFrame(comparison_data)
    
    return df_comparison

# 자유도 영향 시각화
fig_df = visualize_degrees_of_freedom()
fig_df.show()

# 자유도별 비교표
df_table = degrees_of_freedom_comparison()
print("""
📊 자유도별 t-분포 특성 비교:
""")
print(df_table.round(3))

print("""

💡 핵심 관찰:

🎯 자유도가 증가할수록:
   • 분산이 1에 가까워진다 (표준정규분포와 같아짐)
   • 임계값이 작아진다 (더 정확한 검정 가능)
   • 분포의 꼬리가 얇아진다

🎯 실용적 의미:
   • 작은 표본(df < 10): t-분포를 반드시 사용해야 함
   • 큰 표본(df > 30): 정규분포 근사 가능
   • df = 1~2: 분산이 무한대 (매우 불안정)
""")


📊 자유도별 t-분포 특성 비교:

   자유도 (df)  평균     분산   표준편차  95% 임계값  99% 임계값
0         1   0    inf    inf   12.706   63.657
1         3   0  3.000  1.732    3.182    5.841
2        10   0  1.250  1.118    2.228    3.169
3        30   0  1.071  1.035    2.042    2.750
4  ∞ (정규분포)   0  1.000  1.000    1.960    2.576


💡 핵심 관찰:

🎯 자유도가 증가할수록:
   • 분산이 1에 가까워진다 (표준정규분포와 같아짐)
   • 임계값이 작아진다 (더 정확한 검정 가능)
   • 분포의 꼬리가 얇아진다

🎯 실용적 의미:
   • 작은 표본(df < 10): t-분포를 반드시 사용해야 함
   • 큰 표본(df > 30): 정규분포 근사 가능
   • df = 1~2: 분산이 무한대 (매우 불안정)



In [5]:
# 🎲 자유도 체험: 인터랙티브 시뮬레이션
def interactive_degrees_of_freedom_demo():
    "자유도의 의미를 체험할 수 있는 인터랙티브 데모"
    
    @interact(
        df=IntSlider(min=1, max=30, step=1, value=5, description='자유도 (df)'),
        show_comparison=widgets.Checkbox(value=True, description='정규분포와 비교')
    )
    def plot_t_vs_normal(df, show_comparison):
        x = np.linspace(-4, 4, 1000)
        
        fig = go.Figure()
        
        # t-분포
        y_t = stats.t.pdf(x, df)
        fig.add_trace(
            go.Scatter(
                x=x, y=y_t,
                mode='lines',
                name=f't-분포 (df={df})',
                line=dict(width=3, color='red')
            )
        )
        
        if show_comparison:
            # 표준정규분포
            y_normal = stats.norm.pdf(x, 0, 1)
            fig.add_trace(
                go.Scatter(
                    x=x, y=y_normal,
                    mode='lines',
                    name='표준정규분포',
                    line=dict(width=3, color='blue', dash='dash')
                )
            )
        
        # 95% 신뢰구간 영역 표시
        alpha = 0.05
        t_critical = stats.t.ppf(1 - alpha/2, df)
        
        # 가운데 95% 영역
        x_middle = x[(x >= -t_critical) & (x <= t_critical)]
        y_middle = stats.t.pdf(x_middle, df)
        
        fig.add_trace(
            go.Scatter(
                x=np.concatenate([x_middle, x_middle[::-1]]),
                y=np.concatenate([y_middle, np.zeros(len(y_middle))]),
                fill='toself',
                fillcolor='rgba(0, 255, 0, 0.2)',
                line=dict(width=0),
                name='95% 신뢰영역',
                showlegend=True
            )
        )
        
        # 임계값 선들
        for t_val in [-t_critical, t_critical]:
            fig.add_vline(
                x=t_val,
                line_dash="dash",
                line_color="orange",
                annotation_text=f't = {t_val:.3f}'
            )
        
        # 비교를 위한 정규분포 임계값
        if show_comparison:
            z_critical = 1.96
            for z_val in [-z_critical, z_critical]:
                fig.add_vline(
                    x=z_val,
                    line_dash="dot",
                    line_color="blue",
                    annotation_text=f'Z = {z_val:.3f}'
                )
        
        # 표본크기 정보 추가
        sample_size = df + 1
        fig.update_layout(
            title=f'자유도 {df} (표본크기 n={sample_size})에서의 t-분포',
            xaxis_title='값',
            yaxis_title='확률밀도',
            template='plotly_white',
            height=500
        )
        
        fig.show()
        
        # 통계 정보 출력
        if df > 2:
            variance = df / (df - 2)
            std = np.sqrt(variance)
        else:
            variance = "∞"
            std = "∞"
        
        normal_critical = 1.96
        difference = abs(t_critical - normal_critical)
        
        print(f"""
📊 자유도 {df} (표본크기 n={sample_size}) 분석:

📈 분산: {variance}
📏 표준편차: {std}
🎯 95% 임계값: ±{t_critical:.3f}
📊 정규분포 임계값: ±{normal_critical:.3f}
📐 차이: {difference:.3f}

💡 해석:
{'🎯 t-분포가 정규분포보다 넓습니다 (더 보수적)' if difference > 0.1 else '✅ t-분포가 정규분포에 거의 근사합니다'}
{'⚠️ 작은 표본에서는 t-분포를 반드시 사용해야 합니다!' if difference > 0.5 else ''}
""")

# 인터랙티브 데모 실행
interactive_degrees_of_freedom_demo()

interactive(children=(IntSlider(value=5, description='자유도 (df)', max=30, min=1), Checkbox(value=True, descript…

## ⚔️ 4. Z-검정 vs t-검정: 결정적 대결

이제 우리는 두 세계를 비교할 수 있습니다:
- **완벽한 세상** (Z-검정): σ를 알고 있음
- **현실 세상** (t-검정): σ를 모르고 s를 사용

### ⚡ 핵심 차이점

| 특성 | Z-검정 | t-검정 |
|------|--------|--------|
| **모집단 표준편차** | σ 알려짐 ✅ | σ 모름 ❌ |
| **사용하는 표준편차** | σ (정확한 값) | s (추정값) |
| **검정통계량 분포** | 표준정규분포 N(0,1) | t-분포 (df=n-1) |
| **임계값** | 고정 (예: ±1.96) | df에 따라 변함 |
| **표본크기 의존성** | 무관 | 작을수록 더 보수적 |
| **현실 적용성** | 제한적 🚨 | 매우 높음 ✨ |

### 🎯 **가설검정 완전정복: 5단계 과정**

#### **📋 STEP 1: 가설 설정**
```
H₀ (귀무가설): μ = μ₀  ← "차이가 없다"
H₁ (대립가설): μ ≠ μ₀  ← "차이가 있다"
유의수준: α = 0.05
```

#### **📊 STEP 2: 통계량 계산**

**🔵 Z-검정 (σ 알려진 경우)**:
```
Z = (표본평균 - μ₀) / (σ / √n)
```

**🔴 t-검정 (σ 모르는 경우)**:
```
t = (표본평균 - μ₀) / (s / √n)
```

#### **⚠️ 핵심 포인트**:
- **분자**: (실제관측 - 가설값)의 **차이**
- **분모**: 표준오차 (불확실성 크기)
- **결과**: 표준화된 차이의 크기

#### **📈 STEP 3: p-값 계산**

**🔵 Z-검정**:
```python
p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))
# 표준정규분포 N(0,1)에서 z값이 나올 확률
```

**🔴 t-검정**:
```python
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=n-1))
# t-분포(중심=0)에서 t값이 나올 확률
```

#### **💡 p-값의 의미**:
> **"귀무가설(차이없음)이 참이라면, 현재 관측된 결과(또는 더 극단적)가 나올 확률"**

#### **🎯 STEP 4: 판단 기준**
```
if p_value < 0.05:
    print("유의함! H₀ 기각 → 차이가 있다")
else:
    print("유의하지 않음. H₀ 채택 → 차이가 없다")
```

#### **📊 STEP 5: 결론 해석**
- **p < 0.05**: "귀무가설 하에서는 이런 결과가 나올 가능성이 매우 낮다 → 귀무가설이 거짓일 것"
- **p ≥ 0.05**: "귀무가설 하에서도 충분히 나올 수 있는 결과 → 귀무가설을 믿자"

### 🔥 **완벽한 예시: 처음부터 끝까지**

**🎯 상황**: 새로운 교육방법이 효과가 있는가?
- 기존 평균 점수: 75점
- 새 방법으로 10명 교육: [78, 82, 77, 80, 75, 83, 79, 81, 76, 84]

#### **STEP 1: 가설설정**
```
H₀: μ = 75 (새 방법도 기존과 같음)
H₁: μ ≠ 75 (새 방법이 다름)
α = 0.05
```

#### **STEP 2: 통계량 계산**
```python
표본평균 = 79.5
표본표준편차 s = 3.24
n = 10

t = (79.5 - 75) / (3.24 / √10) = 4.5 / 1.02 = 4.41
```

#### **STEP 3: p-값 계산**
```python
p = 2 * (1 - stats.t.cdf(4.41, df=9))
p = 0.0015
```

#### **STEP 4: 판단**
```
p = 0.0015 < 0.05 → 유의함!
```

#### **STEP 5: 결론**
> **"새 교육방법이 기존과 차이가 있다! (p=0.0015)"**

### 🤔 **귀무가설이 어디에 들어가는가?**

1. **통계량 계산시**: 분자에서 비교 기준 (μ₀)
2. **p-값 계산시**: 분포의 가정 ("차이가 없다면")
3. **판단 기준**: 기각할지 채택할지의 대상

### 📊 **분포와 가설의 관계**

#### **통계량 분포의 중심이 항상 0인 이유**:
```
귀무가설이 참 → (표본평균 - μ₀) ≈ 0 
                → 통계량 ≈ 0
```

#### **p-값이 작다는 의미**:
```
"귀무가설이 참이라면 이런 결과는 매우 드물어야 하는데...
 실제로는 이런 결과가 나왔다 → 귀무가설이 거짓일 것"
```

### 🔄 검정통계량 공식 비교

**Z-검정**:
$$Z = \frac{\bar{X} - \mu_0}{\sigma/\sqrt{n}} \sim N(0,1)$$

**t-검정**:
$$t = \frac{\bar{X} - \mu_0}{s/\sqrt{n}} \sim t_{n-1}$$

### 💡 핵심 차이점

**계산의 차이**: σ vs s (단 하나의 차이!)
**결과의 차이**: 
- Z는 표준정규분포 사용 → 임계값 항상 ±1.96
- t는 t-분포 사용 → 임계값이 표본크기에 따라 변함

### 🏆 **최종 정리**

> **가설검정 = "귀무가설 vs 관측 데이터"의 대결**
> 
> 1. 귀무가설을 기준으로 통계량 계산
> 2. 귀무가설이 참이라는 가정하에 p-값 계산  
> 3. p-값이 작으면 귀무가설을 의심
> 4. 0.05보다 작으면 귀무가설 기각

차이는 단 하나: **σ vs s**. 하지만 이 작은 차이가 **통계학 혁명**을 일으켰습니다!

In [ ]:
# ⚔️ Z-검정 vs t-검정 비교 시뮬레이션
def z_vs_t_test_comparison(true_mean=100, true_std=15, hypothesized_mean=95, 
                          sample_sizes=[5, 10, 20, 50], alpha=0.05, n_simulations=1000):
    "Z-검정과 t-검정의 성능 비교"
    
    np.random.seed(42)
    
    results = []
    
    for n in sample_sizes:
        z_rejections = 0
        t_rejections = 0
        z_p_values = []
        t_p_values = []
        z_statistics = []
        t_statistics = []
        
        for _ in range(n_simulations):
            # 표본 생성
            sample = np.random.normal(true_mean, true_std, n)
            sample_mean = np.mean(sample)
            sample_std = np.std(sample, ddof=1)
            
            # Z-검정 (σ 알려진 경우)
            z_stat = (sample_mean - hypothesized_mean) / (true_std / np.sqrt(n))
            z_p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))  # 양측검정
            z_reject = z_p_value < alpha
            
            # t-검정 (σ 모르는 경우)
            t_stat = (sample_mean - hypothesized_mean) / (sample_std / np.sqrt(n))
            t_p_value = 2 * (1 - stats.t.cdf(abs(t_stat), n-1))  # 양측검정
            t_reject = t_p_value < alpha
            
            # 결과 수집
            z_rejections += z_reject
            t_rejections += t_reject
            z_p_values.append(z_p_value)
            t_p_values.append(t_p_value)
            z_statistics.append(z_stat)
            t_statistics.append(t_stat)
        
        # 검정력 계산 (귀무가설이 거짓일 때 기각하는 비율)
        z_power = z_rejections / n_simulations
        t_power = t_rejections / n_simulations
        
        results.append({
            'sample_size': n,
            'z_power': z_power,
            't_power': t_power,
            'z_p_values': z_p_values,
            't_p_values': t_p_values,
            'z_statistics': z_statistics,
            't_statistics': t_statistics
        })
    
    return results

def visualize_z_vs_t_comparison(comparison_results):
    "Z-검정 vs t-검정 비교 결과 시각화"
    
    n_samples = len(comparison_results)
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            '1. 검정력 비교: 실제 차이를 얼마나 잘 찾아내는가?',
            '2. p-값 분포: 1000번 실험의 p-값들 (n=5)',
            '3. 통계량 분포: 계산된 Z값/t값들 vs 이론 (n=5)',
            '4. 임계값 차이: 유의성 판단 기준선'
        ]
    )
    
    # 1. 검정력 비교
    sample_sizes = [r['sample_size'] for r in comparison_results]
    z_powers = [r['z_power'] for r in comparison_results]
    t_powers = [r['t_power'] for r in comparison_results]
    
    fig.add_trace(
        go.Scatter(
            x=sample_sizes,
            y=z_powers,
            mode='lines+markers',
            name='Z-검정 검정력',
            line=dict(color='blue', width=3),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=sample_sizes,
            y=t_powers,
            mode='lines+markers',
            name='t-검정 검정력',
            line=dict(color='red', width=3),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    # 2. p-값 분포 (n=5)
    small_sample_result = comparison_results[0]  # n=5
    
    fig.add_trace(
        go.Histogram(
            x=small_sample_result['z_p_values'],
            nbinsx=30,
            name='Z-검정 p-값',
            marker_color='blue',
            opacity=0.6
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Histogram(
            x=small_sample_result['t_p_values'],
            nbinsx=30,
            name='t-검정 p-값',
            marker_color='red',
            opacity=0.6
        ),
        row=1, col=2
    )
    
    # 유의수준 선
    fig.add_vline(
        x=0.05,
        line_dash="dash",
        line_color="black",
        annotation_text="α = 0.05 (유의성 기준)",
        row=1, col=2
    )
    
    # 3. 통계량 분포 (n=5)
    fig.add_trace(
        go.Histogram(
            x=small_sample_result['z_statistics'],
            nbinsx=40,
            name='실제 Z-통계량들',
            marker_color='blue',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Histogram(
            x=small_sample_result['t_statistics'],
            nbinsx=40,
            name='실제 t-통계량들',
            marker_color='red',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    # 이론적 분포 곡선 추가
    x_range = np.linspace(-6, 6, 200)
    y_normal = stats.norm.pdf(x_range, 0, 1)
    y_t = stats.t.pdf(x_range, 4)  # df = n-1 = 4
    
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_normal,
            mode='lines',
            name='이론적 표준정규분포',
            line=dict(color='blue', width=2, dash='dash')
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_t,
            mode='lines',
            name='이론적 t-분포 (df=4)',
            line=dict(color='red', width=2, dash='dash')
        ),
        row=2, col=1
    )
    
    # 4. 임계값 차이
    z_critical = 1.96  # 95% 양측검정
    t_criticals = [stats.t.ppf(0.975, n-1) for n in sample_sizes]
    differences = [t_crit - z_critical for t_crit in t_criticals]
    
    fig.add_trace(
        go.Scatter(
            x=sample_sizes,
            y=[z_critical] * len(sample_sizes),
            mode='lines',
            name='Z 임계값 (항상 1.96)',
            line=dict(color='blue', width=3, dash='dash')
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=sample_sizes,
            y=t_criticals,
            mode='lines+markers',
            name='t 임계값 (표본크기별)',
            line=dict(color='red', width=3),
            marker=dict(size=8)
        ),
        row=2, col=2
    )
    
    # 레이아웃
    fig.update_layout(
        title='⚔️ Z-검정 vs t-검정: 완벽한 세상 vs 현실 세상',
        height=800,
        showlegend=True
    )
    
    # 축 레이블
    fig.update_xaxes(title_text="표본크기", row=1, col=1)
    fig.update_yaxes(title_text="검정력 (높을수록 좋음)", row=1, col=1)
    fig.update_xaxes(title_text="p-값", row=1, col=2)
    fig.update_yaxes(title_text="빈도", row=1, col=2)
    fig.update_xaxes(title_text="통계량 값", row=2, col=1)
    fig.update_yaxes(title_text="확률밀도", row=2, col=1)
    fig.update_xaxes(title_text="표본크기", row=2, col=2)
    fig.update_yaxes(title_text="임계값 (클수록 보수적)", row=2, col=2)
    
    return fig, differences

# 시뮬레이션 설정 설명
print("🎯 Z-검정 vs t-검정 성능 비교 시뮬레이션")
print("=" * 50)
print("📋 실험 설정:")
print("   • 실제 평균 = 100 (진실)")
print("   • 가설 평균 = 95 (우리가 테스트하는 값)")
print("   • 실제 표준편차 σ = 15")
print("   • 실제로는 차이가 있음! (100 ≠ 95)")
print("")
print("🔬 각 표본크기별로 1000번씩 실험:")
print("   1. 표본 생성 → 2. Z/t 검정 → 3. p < 0.05면 '차이있다' 판단")
print("   4. 1000번 중 몇 번이나 올바르게 '차이있다'고 했는지 = 검정력")
print("")

comparison_results = z_vs_t_test_comparison(
    true_mean=100, true_std=15, hypothesized_mean=95,
    sample_sizes=[5, 10, 20, 50], n_simulations=1000
)

fig_comparison, critical_differences = visualize_z_vs_t_comparison(comparison_results)
fig_comparison.show()

# 상세한 그래프 해석
print("📊 그래프별 상세 해석:")
print("=" * 50)

print("\n🎯 1. 검정력 비교 (왼쪽 위)")
print("   • 검정력 = 실제 차이가 있을 때 그것을 올바르게 발견하는 능력")
print("   • 높을수록 좋음 (1.0 = 100% 발견)")
print("   • Z-검정이 약간 우세한 이유: σ를 정확히 알기 때문")
print("   • 표본크기가 클수록 둘 다 성능 향상")

print("\n📈 2. p-값 분포 (오른쪽 위)")
print("   • 1000번 실험에서 나온 p-값들의 히스토그램")
print("   • 세로선(α=0.05): 이 선 왼쪽이면 '유의하다' 판단")
print("   • 왼쪽에 많이 몰릴수록 = 차이를 잘 발견 = 검정력 높음")
print("   • t-검정의 p-값이 더 오른쪽 → 더 보수적 판단")

print("\n📊 3. 통계량 분포 (왼쪽 아래)")
print("   • 실선: 1000번 계산한 실제 Z/t 통계량들")
print("   • 점선: 이론적으로 예상되는 분포")
print("   • 목적: 실제와 이론이 일치하는지 검증")
print("   • t-분포가 표준정규분포보다 넓음을 확인")

print("\n⚖️ 4. 임계값 차이 (오른쪽 아래)")
print("   • 임계값 = '유의하다'고 판단하는 기준선")
print("   • Z-검정: 항상 ±1.96 (고정)")
print("   • t-검정: 표본크기에 따라 변함 (작을수록 더 큰 값)")
print("   • t-검정이 더 보수적 → 더 확실해야 유의하다고 판단")

# 결과 요약
print("\n🏆 승부 결과 분석:")
print("=" * 50)

for i, result in enumerate(comparison_results):
    n = result['sample_size']
    z_power = result['z_power']
    t_power = result['t_power']
    diff = critical_differences[i]
    
    print(f"\n📊 표본크기 n = {n}:")
    print(f"    🎯 Z-검정 검정력: {z_power:.3f} ({z_power*100:.1f}%가 차이 발견)")
    print(f"    🎯 t-검정 검정력: {t_power:.3f} ({t_power*100:.1f}%가 차이 발견)")
    print(f"    📐 임계값 차이: {diff:.3f} (t가 더 보수적)")
    print(f"    🏅 승자: {'Z-검정' if z_power > t_power else 't-검정' if t_power > z_power else '무승부'}")

print(f"\n💡 핵심 통찰:")
print("=" * 50)
print("🎯 검정력 관점: Z-검정이 약간 우세 (σ를 정확히 알기 때문)")
print("🎯 현실성 관점: t-검정이 압승 (σ를 모르는 것이 현실)")
print("🎯 작은 표본: t-검정의 보수적 접근이 더 안전")
print("🎯 큰 표본: 두 검정의 차이가 거의 없어짐")
print("")
print("🏆 결론: 현실에서는 t-검정이 왕이다! 👑")
print("    σ를 모르는 상황이 99.9%이므로 t-검정을 사용하는 것이 올바름")

## 🎉 5. Gosset의 유산: 1908년의 혁명

### 🌍 통계학계에 미친 영향

William Gosset의 발견은 단순한 수학적 호기심이 아니었습니다. **실용적 필요**에서 나온 혁신이었죠!

#### 🚀 즉각적인 영향
- **농업 연구**: 작은 구획에서의 작물 실험
- **의학 연구**: 제한된 환자 수로도 약물 효과 검증
- **품질 관리**: 비용 효율적인 제품 검사
- **사회과학**: 설문조사와 심리학 실험

#### 📈 장기적인 유산
- **현대 임상시험의 기초**
- **A/B 테스트의 통계적 근거**
- **머신러닝에서의 모델 검증**
- **데이터 사이언스의 필수 도구**

### 🎭 "Student"에서 "Gosset"으로

1922년, 기네스가 정책을 바꾸면서 Gosset은 비로소 실명으로 활동할 수 있게 되었습니다. 하지만 이미 **"Student's t-distribution"**이라는 이름은 영원히 역사에 새겨졌습니다.

### 💭 Gosset의 철학

> "이론을 위한 이론이 아니라, **실제 문제 해결을 위한 통계학**이어야 한다"  
> - William Sealy Gosset

이 철학이 바로 **현대 응용통계학의 정신**입니다!

In [7]:
# 🎉 Gosset의 유산: 현대적 응용 사례
def modern_applications_of_t_test():
    "t-검정의 현대적 응용 사례들을 시뮬레이션"
    
    # 1. 임상시험: 신약 효과 검증
    print("🏥 사례 1: 신약 임상시험")
    print("상황: 새로운 혈압약의 효과를 12명의 환자로 테스트")
    
    np.random.seed(42)
    # 치료 전후 혈압 차이 (실제로는 5mmHg 감소 효과)
    blood_pressure_reduction = np.random.normal(5, 3, 12)
    
    # 일표본 t-검정: 평균 감소량이 0보다 큰가?
    t_stat, p_value = stats.ttest_1samp(blood_pressure_reduction, 0)
    
    print(f"""
📊 결과:

평균 혈압 감소: {np.mean(blood_pressure_reduction):.2f} mmHg

t-통계량: {t_stat:.3f}

p-값: {p_value:.4f}

결론: {'신약에 효과가 있습니다!' if p_value < 0.05 else '효과가 명확하지 않습니다.'}

""")
    
    # 2. A/B 테스트: 웹사이트 전환율
    print("\n💻 사례 2: 웹사이트 A/B 테스트")
    print("상황: 새로운 버튼 디자인이 클릭률을 높이는가?")
    
    # 기존 디자인 vs 새 디자인 (각각 20명씩)
    old_design_clicks = np.random.binomial(1, 0.12, 20)  # 12% 클릭률
    new_design_clicks = np.random.binomial(1, 0.18, 20)  # 18% 클릭률
    
    # 독립 t-검정
    t_stat_ab, p_value_ab = stats.ttest_ind(new_design_clicks, old_design_clicks)
    
    print(f"""
📊 결과:

기존 디자인 클릭률: {np.mean(old_design_clicks):.1%}

새 디자인 클릭률: {np.mean(new_design_clicks):.1%}

t-통계량: {t_stat_ab:.3f}

p-값: {p_value_ab:.4f}

결론: {'새 디자인이 더 효과적입니다!' if p_value_ab < 0.05 else '차이가 명확하지 않습니다.'}

""")
    
    # 3. 교육 연구: 학습 방법 비교
    print("\n📚 사례 3: 교육 방법 효과 연구")
    print("상황: 새로운 교육 방법이 학생들의 성적을 향상시키는가?")
    
    # 동일 학생들의 교육 전후 점수 (대응표본)
    before_scores = np.random.normal(75, 10, 15)
    improvement = np.random.normal(8, 5, 15)  # 평균 8점 향상
    after_scores = before_scores + improvement
    
    # 대응표본 t-검정
    t_stat_paired, p_value_paired = stats.ttest_rel(after_scores, before_scores)
    
    print(f"""
📊 결과:

교육 전 평균: {np.mean(before_scores):.1f}점

교육 후 평균: {np.mean(after_scores):.1f}점

평균 향상: {np.mean(improvement):.1f}점

t-통계량: {t_stat_paired:.3f}

p-값: {p_value_paired:.4f}

결론: {'새 교육 방법이 효과적입니다!' if p_value_paired < 0.05 else '효과가 명확하지 않습니다.'}

""")
    
    return {
        'clinical': (t_stat, p_value),
        'ab_test': (t_stat_ab, p_value_ab),
        'education': (t_stat_paired, p_value_paired)
    }

# 현대적 응용 사례 실행
modern_results = modern_applications_of_t_test()

print("""

🌟 Gosset의 1908년 발견이 오늘날 어떻게 활용되는지 보셨나요?

🎯 핵심 포인트:
   • 작은 표본으로도 과학적 결론 도출 가능
   • 비용 효율적인 실험 설계
   • 불확실성을 정량화하여 신뢰할 수 있는 의사결정

🍺 기네스 맥주 한 잔의 품질 고민이 현대 과학의 기초가 되었습니다!""")

🏥 사례 1: 신약 임상시험
상황: 새로운 혈압약의 효과를 12명의 환자로 테스트

📊 결과:

평균 혈압 감소: 5.89 mmHg

t-통계량: 9.135

p-값: 0.0000

결론: 신약에 효과가 있습니다!



💻 사례 2: 웹사이트 A/B 테스트
상황: 새로운 버튼 디자인이 클릭률을 높이는가?

📊 결과:

기존 디자인 클릭률: 10.0%

새 디자인 클릭률: 25.0%

t-통계량: 1.241

p-값: 0.2221

결론: 차이가 명확하지 않습니다.



📚 사례 3: 교육 방법 효과 연구
상황: 새로운 교육 방법이 학생들의 성적을 향상시키는가?

📊 결과:

교육 전 평균: 73.4점

교육 후 평균: 82.3점

평균 향상: 8.9점

t-통계량: 8.119

p-값: 0.0000

결론: 새 교육 방법이 효과적입니다!




🌟 Gosset의 1908년 발견이 오늘날 어떻게 활용되는지 보셨나요?

🎯 핵심 포인트:
   • 작은 표본으로도 과학적 결론 도출 가능
   • 비용 효율적인 실험 설계
   • 불확실성을 정량화하여 신뢰할 수 있는 의사결정

🍺 기네스 맥주 한 잔의 품질 고민이 현대 과학의 기초가 되었습니다!


## 📚 핵심 개념 요약

### ✨ 오늘의 여행에서 배운 것들

1. **작은 표본의 문제** 🔬
   - σ를 모르고 s를 사용할 때의 추가적 불확실성
   - 비용과 시간 제약으로 인한 현실적 한계

2. **Student's t-분포의 탄생** 🍺
   - William Gosset의 실용적 필요에서 출발
   - 몬테카를로 시뮬레이션으로 직접 유도 가능

3. **자유도의 의미** 🔢
   - df = n-1 (독립적으로 변할 수 있는 편차의 개수)
   - 자유도가 분포의 형태를 결정

4. **Z-검정 vs t-검정** ⚔️
   - 완벽한 세상 vs 현실 세상
   - t-검정이 더 보수적이지만 현실적

5. **현대적 응용** 🚀
   - 임상시험, A/B 테스트, 교육 연구 등
   - 작은 표본으로도 과학적 결론 도출

### 🔑 핵심 공식들

- **t-검정 통계량**: $t = \frac{\bar{X} - \mu_0}{s/\sqrt{n}} \sim t_{n-1}$
- **자유도**: $df = n - 1$
- **t-분포 분산**: $\text{Var}(t) = \frac{df}{df-2}$ (df > 2일 때)

---

## 🧩 연습 문제

### 문제 1: 자유도 이해
표본크기가 8일 때, t-분포의 자유도는 얼마이고, 95% 양측검정의 임계값은?

### 문제 2: Gosset의 상황 재현
맥주 4병의 알코올 도수가 [4.8, 5.2, 4.9, 5.1]%였습니다. 목표 도수 5.0%와 차이가 있는지 t-검정으로 확인하세요.

### 문제 3: Z vs t 비교
동일한 데이터로 Z-검정(σ=0.2 가정)과 t-검정을 수행했을 때 어떤 차이가 있을까요?

---

## 🚀 다음 여행지: "t-test 삼총사"

이제 우리는 t-분포의 비밀을 알았습니다! 🎉

다음 노트북에서는 **세 가지 다른 상황**에서 사용하는 t-검정들을 만나보겠습니다:

- **One-sample t-test**: "우리 제품이 기준을 만족하는가?"
- **Independent t-test**: "A와 B 그룹이 정말 다른가?"
- **Paired t-test**: "치료 전후 차이가 있는가?"

각각 언제, 어떻게 사용하는지 실제 사례와 함께 탐구해보겠습니다!

**다음 노트북**: `03_types_of_t_tests.ipynb`